In [18]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import concurrent.futures
import pandas as pd
import time

Сбор характеристик недвижимости через CianParser

In [ ]:
import cianparser

spisochek = ["Верея", "Высоковск", "Дрезна", "Талдом", "Руза", "Краснозаводск", 
"Пересвет", "Яхрома", "Голицыно", "Волоколамск", "Рошаль", "Кубинка", 
"Куровское", "Пущино", "Электроугли", "Черноголовка", "Хотьково", 
"Звенигород", "Бронницы", "Электрогорск", "Зарайск", "Старая Купавна", 
"Озёры", "Лосино-Петровский", "Красноармейск", "Ликино-Дулёво", "Можайск", 
"Луховицы", "Дедовск", "Апрелевка", "Шатура", "Истра", "Протвино", 
"Краснознаменск", "Кашира", "Котельники", "Солнечногорск", "Дзержинский", 
"Лыткарино", "Фрязино", "Павловский Посад", "Наро-Фоминск", "Ступино", 
"Дмитров", "Чехов", "Егорьевск", "Дубна", "Видное", "Клин", "Ивантеевка", 
"Лобня", "Воскресенск", "Сергиев Посад", "Ногинск", "Жуковский", "Пушкино", 
"Реутов", "Долгопрудный", "Орехово-Зуево", "Раменское", "Щёлково", 
"Серпухов", "Одинцово", "Домодедово", "Коломна", "Электросталь", 
"Красногорск", "Люберцы", "Королёв", "Мытищи", "Химки", "Подольск", "Балашиха",
"Москва"]

# Сбор недвижимости по каждому из количества комнат
for ii in spisochek:
    pars = cianparser.CianParser(location=ii)
    for i in range(1, 6):
        data = pars.get_flats(
            deal_type="sale",
            with_extra_data = True,
            rooms=(i), 
            with_saving_csv=True, 
            additional_settings={
                "start_page": 1, 
                "end_page": 54
            })

        print(data[0])

In [ ]:
import glob
import pandas as pd

# Папка с CSV файлами
path = "C:/Users/User/Desktop/parser/"
filenames = glob.glob(path + "/*.csv")

# Пустой датафрейм для хранения данных
dfs = []

# Перебираем все файлы в папке
for filename in filenames:
    df = pd.read_csv(filename, usecols=[0], error_bad_lines=False)
    dfs.append(df)

# Объединение
combined_data = pd.concat(dfs)

# Сохранение объединенного датасета в новый файл b_data.csv
combined_data.to_csv('b_data.csv', index=False)

Сбор дополнительных значений недвижимости

In [36]:
start = time.time()
# файл для хранения информации
rows_f = open('info.txt', 'a')
rows_f.write("url,living_meters,kitchen_meters,y_c\n")

err = open('errors.txt', 'a')  # файл хранящий возникающие ошибки
success = 0

# Параметры для requests.get()
st_accept = 'text/html'
st_useragent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15'
TIMEOUT = 25
headers = {
    "Accept": st_accept,
    "User-Agent": st_useragent
}

In [ ]:
def convert_to_number(s):
    s = s.replace(",", ".")  # замена запятой на точку для float
    try:
        return int(s)
    except ValueError:
        try:
            return float(s)
        except ValueError:
            return s
        

def remove_non_numeric_values(lst):
    return [x for x in lst if isinstance(x, (int, float))] # удаление нечисловых значений


def get_info_from_link(url_value): # получение характеристик по ссылке
    global b, success 
    
    # Отправка запроса 
    time.sleep(7)
    response = requests.get(url_value, headers=headers, timeout=TIMEOUT)
    print(response)
    if response.status_code == 200:
        success += 1

    html = response.content

    # Анализ контекнта с помощью BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Поиск тегов с классом
    p_tags = soup.find_all('p', class_='a10a3f92e9--color_black_100--Ephi7')

    # Извлечение характеристик
    characteristics = []
    for p_tag in p_tags:
        characteristic = p_tag.text.strip()  # извлечение текста
        characteristics.append(characteristic)  # добавление в список

    # удаление нечисловых значений
    characteristics = remove_non_numeric_values([convert_to_number(c) for c in characteristics])


    # удаление значений после даты постройки
    for i, item in enumerate(characteristics):
        if isinstance(item, int) and 1000 <= item <= 9999:
            del characteristics[i+1:]
            break

    # поиск даты постройки с добвлением в столбец year_of_construction
    y_c = next((item for item in characteristics if isinstance(item, int) and 1000 <= item <= 3000), 0)


    # поиск жилой площади и кухонной среди всех значений
    if len(characteristics) < 3:
        first_value = 0
        second_value = 0
        third_value = 0
    else:
        first_value = float(characteristics[0])
        second_value = float(characteristics[1])
        third_value = float(characteristics[2])

        if first_value > second_value + third_value:
            living_meters = second_value
            kitchen_meters = third_value
        else:
            living_meters = second_value
            kitchen_meters = 0

        # создание новой строки
        new_row = [url_value, str(living_meters), str(kitchen_meters), str(y_c)]

        b += 1
        print(b)

        return new_row


data = pd.read_csv('b_data.csv')
urls = data['url'].tolist()[:]
b = 0


# Сбор данных в 6 параллельных потоков
err_counter = 0
CONNECTIONS = 6  # кол-во потоков
with concurrent.futures.ThreadPoolExecutor(max_workers=CONNECTIONS) as executor:
    future_to_url = (executor.submit(get_info_from_link, url) for url in urls)
    for future in tqdm(concurrent.futures.as_completed(future_to_url), total=len(urls)):
        try:
            info = future.result()
            add_row = True
        except Exception as exc:
            add_row = False
            err_counter += 1
            err.write(str(exc) + "\n")
        finally:
            if add_row and info:
                print(info)
                rows_f.write(",".join(info) + "\n")
            else:
                pass

end = time.time()

print("The time of execution of above program is :",
      (end - start) * 10 ** 3, "ms")

rows_f.close()
err.close()
print(f"Ошибок: {err_counter}")
print('Успехов:', success)

Добавление дополнительных характеристик в основной датасет

In [ ]:
df1 = pd.read_csv('info_.csv')
df2 = pd.read_csv('b_data.csv')


# Поочередное сравнение url и присваивание значений из df1 в df2
for index, row in df1.iterrows():
    url = row['url']
    mask = df2['url'] == url

    if mask.any():
        df2.loc[mask, 'living_meters'] = row['living_meters']
        df2.loc[mask, 'kitchen_meters'] = row['kitchen_meters']
        df2.loc[mask, 'year_of_construction'] = row['year_of_construction']
    

df2.to_csv('b_data.csv', index=False)

Сбор стоимости квартир по ссылкам

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Загрузка датасета
data = pd.read_csv('b_data.csv')

# Датасет для записи
data1 = pd.DataFrame(columns=['url', 'price'])

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
price_classes = ["a10a3f92e9--amount--v3ROW", "a10a3f92e9--amount--ON6i1"]

# Перебор ссылок
for index, row in data.iterrows():
    url = row['url']
    for price_class in price_classes:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        price_element = soup.find("div", class_=price_class)
        
        if price_element is not None:
            price_text = price_element.text.strip()
            price_text = int(''.join(filter(str.isdigit, price_text)))
            
            # Заполнение построчно
            new_row = pd.DataFrame({'url': [url], 'price': [price_text]})
            data1 = pd.concat([data1, new_row], ignore_index=True)
            
            print(f"Ссылка: {url}, Цена: {price_text}")
            break
    else:
        print(f"Ссылка: {url}, Цена: не найдена")
        
# Экспорт
data1.to_csv('apr17.csv', index=False)

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Загрузка датасета
data = pd.read_csv('b_data.csv')

# Датасет для записи
data1 = pd.DataFrame(columns=['url', 'price'])

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
price_classes = ["a10a3f92e9--amount--v3ROW", "a10a3f92e9--amount--ON6i1"]

# Перебор ссылок
for index, row in data.iterrows():
    url = row['url']
    for price_class in price_classes:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        price_element = soup.find("div", class_=price_class)
        
        if price_element is not None:
            price_text = price_element.text.strip()
            price_text = int(''.join(filter(str.isdigit, price_text)))
            
            # Заполнение построчно
            new_row = pd.DataFrame({'url': [url], 'price': [price_text]})
            data1 = pd.concat([data1, new_row], ignore_index=True)
            
            print(f"Ссылка: {url}, Цена: {price_text}")
            break
    else:
        print(f"Ссылка: {url}, Цена: не найдена")
        
# Экспорт
data1.to_csv('may25.csv', index=False)

Добавление изменения цен в основной датасет

In [13]:
import pandas as pd

df1 = pd.read_csv('b_data.csv')
df2 = pd.read_csv('apr17.csv')
df3 = pd.read_csv('may25.csv')

# Переименовывание
df1.rename(columns={'price': 'price_feb'}, inplace=True)
df2.rename(columns={'price': 'price_apr'}, inplace=True)
df3.rename(columns={'price': 'price_may'}, inplace=True)

# Сравнение url и добавление значений
for index, row in df1.iterrows():
    url = row['url']
    if url in df2['url'].values:
        df1.loc[index, 'price_apr'] = df2.loc[df2['url'] == url, 'price_apr'].values[0]
    
    if url in df3['url'].values:
        df1.loc[index, 'price_may'] = df3.loc[df3['url'] == url, 'price_may'].values[0]

df1 = df1[['url', 'location', 'floor', 'floors_count', 
           'rooms_count', 'living_meters', 'kitchen_meters', 'total_meters','price_feb', 'price_apr', 
           'price_may', 'district', 'street',
           'underground', 'residential_complex', 'year_of_construction']]

# Сохранение измененного датасета
df1.to_csv('b_data.csv', index=False)